In [19]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from datetime import datetime , timedelta

In [20]:
def connect():
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    return driver

In [21]:
def get_links(keyword = "hadoop"):

    Links = []
    Posted = []

    driver = connect()
    url = f'https://fr.indeed.com/jobs?q={keyword}&l=France&fromage=7'
    
    while True:
        try:
            
            driver.get(url)
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            td = soup.find_all('td',{'class':'resultContent'})
            link = []
            posted = []
            for i in td:
                link.append("https://fr.indeed.com/"+i.find('a',{'class':'jcs-JobTitle css-jspxzf eu4oa1w0'}).get('href'))
                posted.append(i.find('span',{'data-testid':'myJobsStateDate'}).text)
            Links.extend(link)
            Posted.extend(posted)
            try:
                url = 'https://fr.indeed.com' + soup.find('a', {'aria-label':'Next Page'}).get('href')
            except:
                break
        except:
            print("Error URL")
            break
    
    print(len(Links),"Jobs found for",keyword)
            
    driver.quit()
    return Links,Posted

In [22]:
def get_all_links(Keywords):
    Links = []
    Posted = []
    for key in Keywords:
        links, posted = get_links(key)
        Links.extend(links)
        Posted.extend(posted)
    return Links, Posted

In [23]:
def get_datas(Keywords):
    List_job = []
    Links, Posted = get_all_links(Keywords)
    for i in range(len(Links)):
        try:
            job = {}
            driver = connect()
            driver.get(Links[i])
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source,'html')
            try:
                job['job_link'] = Links[i]
            except:
                job['job_link'] = None

            try:
                job['job_name'] = soup.find('h1',{'class':'jobsearch-JobInfoHeader-title'}).text.strip()
            except:
                job['job_name'] = None
                
            try:
                job['job_text'] = soup.find('div',{'id':'jobDescriptionText'}).text.strip().replace('\n','')
            except:
                job['job_text'] = None
                
            try:
                job['job_company'] = soup.find('div',{'data-testid' : 'inlineHeader-companyName'}).find('span').text.strip()
            except:
                job['job_company'] = None
                
            try:
                job['job_location'] = soup.find('div',{'id':'jobLocationText'}).text.strip()
            except:
                job['job_location'] = None
            try:
                job['job_type'] =  soup.find('div',{'id':"salaryInfoAndJobType"}).find('span',{'class':'css-k5flys eu4oa1w0'}).text.replace('-','').strip()
            except:
                job['job_type'] = None
                
            try:
                date = Posted[i]
                now = datetime.now()
                if 'instant' in date:
                    formatted_date = now.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '1jour' in date.replace('\xa0', ''):
                    date = now - timedelta(days=1)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '2jour' in date.replace('\xa0', ''):
                    date = now - timedelta(days=2)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '3jour' in date.replace('\xa0', ''):
                    date = now - timedelta(days=3)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '4jour' in date.replace('\xa0', ''):
                    date = now - timedelta(days=4)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '5jour' in date.replace('\xa0', ''):
                    date = now - timedelta(days=5)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '6jour' in date.replace('\xa0', ''):
                    date = now - timedelta(days=6)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '7jour' in date.replace('\xa0', ''):
                    date = now - timedelta(days=7)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
            except:
                job['job_date'] = None
        except:
            print("Error link")
            pass
        
        List_job.append(job)
        driver.quit()
        
        if i%10==0:
            print(i,"jobs extracted!",end='\r')
            
    return pd.DataFrame(List_job)


In [24]:
Keywords = ['data analyst','data scientist','data engineer','web developer','mobile developer']
job_df = get_datas(Keywords)
job_df

182 Jobs found for data analyst
Error URL
180 Jobs found for data scientist
273 Jobs found for data engineer
Error URL
90 Jobs found for web developer
0 Jobs found for mobile developer


,job_link,job_name,job_text,job_company,job_location,job_type,job_date
0,https://fr.indeed.com//rc/clk?jk=f02d26eaa5ee3...,Data Analyst Transport H/F,Bouygues Construction Matériel apporte son sav...,Bouygues Construction Matériel,Chilly-Mazarin (91),CDI,2024-07-10
1,https://fr.indeed.com//rc/clk?jk=0793e3b3707f5...,Data analyst F/H,Détail de l'offre Informations générales Descr...,ALKERA,Clichy (92),"CDI, Temps plein",2024-07-12
2,https://fr.indeed.com//rc/clk?jk=92b3c6db22ecb...,Junior Data Analyst H/F,Lectra SA Junior Data Analyst H/F ...,Lectra,Cestas (33),None,2024-07-11
3,https://fr.indeed.com//rc/clk?jk=965d2d27c6c5e...,Data Analyst (H/F),Lille CDI Publiée il y a plus de ...,Infogene,Lille (59),CDI,2024-07-12
4,https://fr.indeed.com//rc/clk?jk=ed41fb1cfd983...,CDI - Hyperviseur / Data Analyst (F/H),Identifiant de la demande: 31678 Vill...,GRDF,Paris (75),"CDI, Temps plein",2024-07-12
...,...,...,...,...,...,...,...
720,https://fr.indeed.com//rc/clk?jk=97d31604a7f88...,Développeur Web Fullstack Symfony / React H/F,Description du poste Nous recherchons un D...,Septeo,Montpellier (34),CDI,2024-07-05
721,https://fr.indeed.com//rc/clk?jk=e4821a298abc3...,None,None,None,None,None,2024-07-09
722,https://fr.indeed.com//rc/clk?jk=17fc1d3c208e5...,Développeur Front-end Angular (h/f),Start ASAP Duration > 12 monthsemagine recherc...,Skillspark AB,Saint-Denis (93),None,2024-07-09
723,https://fr.indeed.com//rc/clk?jk=046754fa7c04c...,"Développeur Senior NEXT ""fullstack"" (F/H) (IT)...",Notre client est l'expert mondial en qualité d...,ODHCOM - FREELANCEREPUBLIK,Paris (75),Indépendant / freelance,2024-07-06


In [25]:
csv_filename = "indeed.csv"
job_df.to_csv(csv_filename, index=False)
print(f"Le fichier CSV a été créé avec succès sous le nom {csv_filename}.")


Le fichier CSV a été créé avec succès sous le nom indeed.csv.


In [27]:
a=job_df['job_date'][700]
print(a)

2024-07-06


In [28]:
job_df.shape

(725, 7)